In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

Imported all nessessary libraries

In [ ]:
dir(transforms)

All available transforms

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

dataset = torchvision.datasets.CIFAR10(root='./datasets/cifar10/train', download=True, transform = transform)

dataset

dataloader = torch.utils.data.DataLoader(dataset,
                                        batch_size=64,
                                        shuffle=True,
                                        num_workers=2)

data_mean = []
data_std = []

for i, data in enumerate(dataloader, 0):
    
    numpy_image = data[0].numpy()
    
    batch_mean = np.mean(numpy_image, axis=(0, 2, 3))
    batch_std = np.std(numpy_image, axis=(0, 2, 3))
    
    data_mean.append(batch_mean)
    data_std.append(batch_std)
    
data_mean = np.array(data_mean)
data_std = np.array(data_std)

data_mean = data_mean.mean(axis=0)
data_std = data_std.mean(axis=0)

print(data_mean)
print(data_std)

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(),
    transforms.Pad(4),
    transforms.ToTensor(),
    transforms.Normalize(data_mean, data_std)
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(data_mean, data_std)
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./datasets/cifar10/train', 
                                        train=True,
                                        download=True, 
                                        transform = train_transform)

testset = torchvision.datasets.CIFAR10(root='./datasets/cifar10/train', 
                                        train=False,
                                        download=True, 
                                        transform = test_transform)

trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=64,
                                          shuffle=True,
                                          num_workers=2)

testloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=64,
                                          shuffle=False,
                                          num_workers=2)

In [ ]:
class ConvNet(nn.Module):
    
    def __init__(self, config):
        super(ConvNet, self).__init__()
        
        self.config = config
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(num_channels, size_64, kernel_size, padding = 1, bias = False), #40*40
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2), #20*20
            nn.BatchNorm2d(size_64)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(size_64, size_128, kernel_size, padding = 1, bias = False), #20*20
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2), #10*10
            nn.BatchNorm2d(size_128)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(size_128, size_256, kernel_size, padding = 1, bias = False), #10*10
            nn.ReLU()
        )
        
        self.conv4 = nn.Sequential(
            nn.Conv2d(size_256, size_256, kernel_size, padding = 1, bias = False), #10*10
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2), #5*5
            nn.BatchNorm2d(size_256)
        )
        
        self.conv5 = nn.Sequential(
            nn.Conv2d(size_256, size_512, kernel_size, padding = 1, bias = False), #5*5
            nn.ReLU()
        )
        
        self.conv6 = nn.Sequential(
            nn.Conv2d(size_512, size_512, kernel_size, padding = 1, bias = False), #5*5
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2), #2*2
            nn.BatchNorm2d(size_512)
        )
        
        self.conv7 = nn.Sequential(
            nn.Conv2d(size_512, size_512, kernel_size, padding = 1, bias = False), #2*2
            nn.ReLU(),
            #nn.Dropout2d(0.5)
        )
        
        self.conv8 = nn.Sequential(
            nn.Conv2d(size_512, size_512, kernel_size, padding = 1, bias = False), #2*2
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2), #1*1
            nn.BatchNorm2d(size_512),
            #nn.Dropout2d(0.5)
        )
        
        self.linear1 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(size_512, out1_size),
            nn.ReLU(),
            #nn.Dropout(0.75)
        )
        
        self.linear2 = nn.Sequential(
            nn.Linear(out1_size, out2_size),
            nn.ReLU(),
            #nn.Dropout(0.75)
        )
        
        self.linear3 = nn.Sequential(
            nn.Linear(out2_size, out3_size)
        )
        
    def forward(self, x):
        
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.conv5(out)
        out = self.conv6(out)
        out = self.conv7(out)
        out = self.conv8(out)
        
        out = self.linear1(out)
        out = self.linear2(out)
        out = self.linear3(out)

        
        return out    